[17 使用和购买 GPU](https://www.bilibili.com/video/BV1z5411c7C1?spm_id_from=333.999.0.0)

In [1]:
!nvidia-smi

Wed May 11 17:17:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.68.01    Driver Version: 512.59       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 37%   31C    P8    35W / 350W |   1762MiB / 12288MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import nn

In [3]:
torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7f0080249b50>)

In [4]:
# GPU数量查看
torch.cuda.device_count()

1

现在我们定义了两个⽅便的函数，这两个函数允许我们在请求的GPU不存在的情况下运⾏代码。

In [5]:
def try_gpu(i=0): #@save
    """如果存在，则返回gpu(i)，否则返回cpu()。"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): #@save
    """返回所有可⽤的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

- 默认情况下，张量是在CPU上创建的。我们可以查询张量所在的设备
- 需要注意的是，⽆论何时我们要对多个项进⾏操作，它们都必须在同⼀个设备上。例如，如果我们对两个张
量求和，我们需要确保两个张量都位于同⼀个设备上，否则框架将不知道在哪⾥存储结果，甚⾄不知道在哪
⾥执⾏计算。

In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

- 存储在GPU上
- 有⼏种⽅法可以在GPU上存储张量。例如，我们可以在创建张量时指定存储设备。接下来，我们在第⼀个gpu上
创建张量变量X。在GPU上创建的张量只消耗这个GPU的显存。我们可以使⽤nvidia-smi命令查看显存使
⽤情况。⼀般来说，我们需要确保不创建超过GPU显存限制的数据。


In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
# 假设你⾄少有两个GPU，下⾯的代码将在第⼆个GPU上创建⼀个随机张量。
# 没有则依旧默认cpu
Y = torch.rand(2, 3, device=try_gpu(1))
Y.device

device(type='cpu')

In [9]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [10]:
# 必须转移至同意设备方可运行
Y.cuda(0) + Z

tensor([[1.5183, 1.8663, 1.0938],
        [1.2340, 1.4188, 1.0574]], device='cuda:0')

假设变量Z已经存在于第⼆个GPU上。如果我们还是调⽤Z.cuda(1)怎么办？它将返回Z，而不会复制并分
配新内存。

In [11]:
Z.cuda(0) is Z

True

类似地，神经⽹络模型可以指定设备。下⾯的代码将模型参数放在GPU上。

In [12]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)

tensor([[-0.9836],
        [-0.9836]], device='cuda:0', grad_fn=<AddmmBackward0>)

让我们确认模型参数存储在同⼀个GPU上。

In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

: 